In [32]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# O método de Newton

## $ \S 1 $ Descrição do método de Newton

Suponha que desejemos encontrar um zero de uma função _diferenciável_ $ f $.
Partindo de uma estimativa inicial $ x_0 $, a idéia por trás do __método de
Newton__ é substituir a função pela sua reta tangente na estimativa atual
$ x_n $ e tomar a intersecção desta com o eixo-$x$ como o valor da próxima
estimativa $ x_{n + 1} $. 

Em símbolos, a reta tangente ao gráfico de $ f $ em $ x_n $ é descrita pela equação:
\begin{equation*}\label{E:line}
y = f(x_n) + f'(x_n)\,(x - x_{n})\,. \tag{1}
\end{equation*}
A aproximação $ x_{n + 1} $ seguinte é obtida fazendo $ y = 0 $ e encontrando
o valor de $ x $ correspodente:
$$
\boxed{x_{n + 1} = x_n - \frac{f(x_n)}{f'(x_n)}}
$$

📝 A expressão do lado direito em \eqref{E:line} é o polinômio de Taylor de grau
$ 1 $ de $ f $ ao redor de $ x_n $, às vezes chamado nos cursos de Cálculo de
_linearização_ ou _aproximação linear_ de $ f $ aí. Portanto sob outro ponto de
vista, completamente equivalente ao que descrevemos antes, o método de Newton
consiste em substituir a função original $ f $ pelo seu polinômio de Taylor de
grau 1 na estimativa atual e usar o zero deste como nova aproximação para o zero
de $ f $.

📝 O método de Newton também é conhecido como _método de Newton-Raphson_; Joseph
Raphson (1668–1715) foi um matemático inglês contemporâneo de Newton.

__Exemplo 1:__ Rode em seqüência o código nas três células abaixo para ver uma
animação do método de Newton aproximando um zero da função $ f(x) =  x^3 - 2x -
3 \ln x $ em $ 0.70271253 $, aproximadamente. (Para facilitar a navegação
posterior, depois você pode esconder as células clicando sobre a barra
correspondente à esquerda.) 

In [79]:

def print_solution(xs: list[float], ys: list[float], freq: int = 1) -> None:
    """
    Given two arrays xs and ys of the same length, prints a table whose n-th
    line consists of three entries: the values of n, xs[n] and ys[n].
    Parameters:
        * The arrays xs and ys. 
        * A parameter freq used to print only one in every freq line. The
          first and last line are always printed. If freq == 0, then only these
          lines are printed.
    Output: None.
    Prints: A header and the table described above.
    """
    def print_header() -> None:
        """
        Prints the table's header.
        """
        print("\n|       n      ", end="")
        print("    x_n            ", end="")
        print("    f(x_n)      |")
        print("|=================================================|")
        
    def print_line(n: int, x: float, y: float) -> None:
        """
        Pretty-prints n, x and y.
        """
        print(f"|      {n:02}", end="")
        print(f"    {x:15.8f}", end="")
        print(f"    {y:15.8f}   |")
    
    
    N = len(xs)
    if freq == 0:       # If freq == 0, print only first and last lines.
        freq = N - 1
    print_header()
    print_line(0, xs[0], ys[0])
    for n in range(1, N, freq):
        print_line(n, xs[n], ys[n])
    if n != N - 1:
        print_line(n, xs[N], ys[N])
    print("|_________________________________________________|\n")
        
    return None

In [80]:
def newton_animation(f: Callable[[float], float],
                     df: Callable[[float], float],
                     a: float, b :float, x: float,
                     N: int = 4, title: str = "", duration: float = 0.75
                     ) -> tuple[list[float], list[float]]:
    """
    Displays an animation of Newton's method applied to a function.
    Parameters:
        * A differentiable real function f.
        * Its derivative df (as a function).
        * The two endpoints a and b of the interval in the x-axis where the
          animation takes place.
        * An initial estimate x for the zero in [a, b].
        * The maximum number N of iterations.
        * A title to be displayed at the top of the diagram.
        * The duration of the pause between slides of the animation, in seconds.
          Set duration = 0 to produce a figure instead of an animation.
    Output:
        * Two lists xs and ys containing the estimates and the values of the
          function f at each of them.
    Displays:
        * The animation in a pop-up window.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    
    def pause(duration):
        """
        Pauses the animation for duration seconds, provided duration > 0.
        """
        if duration > 0:
            plt.pause(duration)

    def tangent_line_at(x_0):
        return lambda x: f(x_0) + df(x_0) * (x - x_0)
    
    def iteration(x):
        return x - f(x) / df(x)


    cmap = plt.get_cmap("tab10")               # Used to control the colors.
    P = 200                                    # Number of points in each plot.
    width = 1.75                               # Line width.
    marker_size = 5
    domain = np.linspace(a, b, P)              # Generates P nodes from a to b.
    xs = [x]                                   # Stores the estimates.
    for _ in range(N):                         # Filling xs.
        xs.append(iteration(xs[-1]))
    ys = [f(x) for x in xs]                    # Stores f of the estimates.
    tangent_lines = [tangent_line_at(x) for x in xs]

    # Generate sample points for x-intervals between consecutive estimates:
    xs_range = [np.linspace(xs[n], xs[n + 1], P) for n in range(N)]
    # Lists containing the x and y coordinates for plotting vertical lines:
    xs_vert = [np.linspace(xs[n], xs[n], P) for n in range(N)]
    ys_vert = [np.linspace(0, ys[n], P) for n in range(N)]

    # Draw the graph of f:
    plt.axhline(y=0.0, color='black', linestyle='-', lw=width)
    plt.xlabel("$ x $-axis")
    plt.ylabel("$ y $-axis")
    plt.title(title)
    plt.grid(True)
    plt.plot(domain, f(domain), label="$ y = f(x) $", lw=width)
    plt.legend()

    # Mark the initial estimate on the x-axis:
    pause(duration)
    plt.plot(xs[0], 0, color=cmap(1), marker="x",
             mew=width, label=f'$ x_{0} $')

    for n in range(0, N):
        # Mark x_n on the x-axis:
        plt.plot(xs[n], 0, color=cmap(n + 1), marker="x", mew=width)
        pause(duration)
        # Draw the segment of the line x = x_n from y = 0 to y = y_n:
        plt.plot(xs_vert[n], ys_vert[n], linestyle='dotted',
                 lw=width, color='black')
        # Plot (x_n, y_n):
        plt.plot(xs[n], ys[n], color='black', marker="o", ms=marker_size)
        pause(duration)
        # Plot the tangent line at (x_n, f(x_n)):
        plt.plot(xs_range[n], tangent_lines[n](xs_range[n]),
                 linestyle='--', color=cmap(n + 2))
        pause(duration)
        plt.plot(xs[n + 1], 0, color=cmap(n + 2), marker="x",
                 mew=width, label=f'$ x_{n + 1} $')
        plt.legend()
    
    return xs, ys


In [82]:
from numpy import log
a = 0.4
b = 1.5
N = 3
f = lambda x: x**3 - 2 * x - 3 * log(x)
df = lambda x: 3 * x**2 - 2 - 3 / x
x_0 = 1
pause = 1.0
titulo = "Exemplo: Método de Newton para $ y = x^3 - 2x - 3\,\ln x $."\
         "\n Estimativa inicial $ x_0 = 1 $ e $ N = 3 $ iterações."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pause)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         1.00000000        -1.00000000   |
|      01         0.50000000         1.20444154   |
|      02         0.66612987         0.18213330   |
|      03         0.70134213         0.00657080   |
|_________________________________________________|



![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_1.png
"Exemplo de aplicação do método de Newton")

__Problema 1:__ Usando o computador apenas como uma calculadora (e quando
necessário), aplique o método de Newton para aproximar um zero das funções
abaixo, usando $ N = 4 $ iterações e a estimativa inicial $ x_0 $ indicada.
Depois use os procedimentos `newton_animation` e `print_solution` seguindo
o modelo do Exemplo 1 para verificar sua resposta.

(a) $ y = xe^x - 1\, ,\quad  x_0 = 0 $. 

(b) $ y = \arctan x - 1\, ,\quad  x_0 = 1 $. 

(c) $ y = \ln x - 3\, ,\quad  x_0 = 10 $. 

(d) $ y = x^3 - 9x + 3\, ,\quad  x_0 = 3 $.

## $ \S 2 $ Implementação do método de Newton
 
Assim como no método do ponto fixo, a princípio não dispomos de um intervalo que
encaixota um zero nem de uma cota superior para o erro cometido que seja
independente da função. Por causa disto, não há um critério de parada
totalmente satisfatório. Na implementação abaixo, o procedimento é interrompido
assim que _ambas_ as desigualdades abaixo forem satisfeitas pela estimativa $
x_n $ atual:
\begin{equation*}\label{E:error}
\left\vert x_{n} - x_{n - 1} \right\vert < \varepsilon \quad \tag{1}
\end{equation*}
e
\begin{equation*}\label{E:magnitude}
\left\lvert f(x_n) \right\rvert < \varepsilon \,.\tag{2}
\end{equation*}
Como sempre, aqui $ \varepsilon > 0 $ é uma tolerância previamente escolhida.
Infelizmente, \eqref{E:error} não previne que ainda estejamos longe do zero,
pois esta quantidade pode ser pequena se a aproximação for muito lenta. E
\eqref{E:magnitude} tem o defeito de não ser invariante, no sentido que
apesar dos zeros das funções $ f(x) $ e $ kf(x) $ ($ k \ne 0 $ uma constante
real) serem exatamente os mesmos, o critério pode fornecer estimativas muito
diferentes se $ k $ está longe de $ 1 $.

Como a convergência do método de Newton não é garantida, também é recomendável
que o procedimento seja interrompido caso um número máximo de iterações seja
violado; esta condição também está inclusa na implementação abaixo.

Outra dificuldade está relacionada ao denominador $ f'(x_n) $ que aparece na
fórmula para a próxima estimativa. Mesmo que a derivada $ f' $ exista em todo
ponto, se ela for nula ou muito pequena, isto pode gerar um erro de _overflow_.

In [134]:

def newton(f: Callable[[float], float], df: Callable[[float], float],
           x: float, eps: float = 1.0e-3, max_iter: int = 100
           ) -> tuple[list[float], list[float]]:
    """
    Applies Newton's method to try to find a zero of a function.
    Parameters:
        * A differentiable function f of one real variable.
        * Its derivative df (as another function).
        * An initial guess x for a zero.
        * A tolerance eps such that the procedure terminates if both
            |x_n - x_{n - 1}| < eps
          and
            |f(x_n)| < eps
          where x_n is the estimate provided by the current iteration.
        * The maximum allowed number of iterations, max_iter.
    Output:
        * Two lists, xs and ys, containing the estimates and the values
          of f at each of them, respectively.
    Prints:
        * A warning or error message, in case of failure.
        * The last estimate.
        * The value of f at this estimate.
        * The number of iterations that were performed.
    """
    def iterate(x):
        """ Perform one step of Newton's method to yield the next estimate. """
        return x - f(x) / df(x)

    if eps <= 0:       # Error: invalid value for eps.
        raise ValueError("The tolerance must be positive!")
    # Check whether max_iter is a positive integer:
    if not (isinstance(max_iter, int) and max_iter > 0):
        raise ValueError("'max_iter' must be a positive integer!")
    x = float(x)       # Make sure x is of type float.
    xs = [x]           # Create a list to store the estimates.
    ys = [f(x)]        # List to store the values of f at the estimates.
    iterations = 0     # Counter for the number of iterations.
    error = 2 * eps    # Any value > eps will do.
    while error >= eps and iterations < max_iter:
        try:           # Compute the next estimate and check for overflow.
            new_x = iterate(x)
        except OverflowError:
            print("Overflow error! Either the estimates are diverging or"
                  "the derivative at the current estimate is too small.")
            return xs, ys
        else:
            xs.append(new_x)          # Store the new estimate.
            ys.append(f(new_x))       # Store f of the new estimate.
            # Update 'error':
            error = max(abs(xs[-1] - xs[-2]), abs(ys[-1])) 
            x = new_x                 # Update x.
            iterations += 1           # Update iteration counter.
    
    if iterations == max_iter:
        print("Warning: The maximum number of iterations was reached!")
    print(f"After {iterations} iterations, ", end='')
    print(f"the estimate for the fixed point is:\n{xs[-1]:15.8f}\n"
          f"The value of the function at this point is: \n{ys[-1]:15.8f}")
    return xs, ys

## $ \S 3 $ Vantagens e desvantages do método de Newton

### $ 3.1 $ Vantagens do método de Newton
Veremos abaixo que no método de Newton o erro cometido na iteração seguinte é
aproximadamente proporcional ao *quadrado* do erro da iteração atual. Portanto
podemos esperar que uma vez que o erro seja menor que $ 0.1 $, a cada passo o
número de dígitos decimais de precisão aproximadamente dobrará. Dito de outra
forma, o número de dígitos de precisão crescerá _exponencialmente_ com o número
$ n $ de iterações.

Este desempenho deve ser comparado com o do método da bisseção, em que o erro
seguinte é aproximadamente proporcional ao erro atual (por um fator de $ 1/2 $).
Informalmente, isto significa que uma vez que consigamos uma estimativa próxima
o suficiente de um zero, o método de Newton convergirá _muito_ mais rapidamente
que os outros métodos que estudamos.

Outra vantagem é que para aplicar o método de Newton, não precisamos encaixotar
o zero.

Provaremos abaixo que _o método de Newton sempre converge a um zero, contanto
que a estimativa inicial $ x_0 $ seja tomada próxima o suficiente dele_. Porém,
na maioria dos casos é difícil saber de antemão quão próximo é "próximo o
suficiente". Para dirimir esta dificuldade, uma alternativa é analisar o gráfico
da função antes de aplicar o método de Newton e escolher a estimativa inicial o
mais precisamente possível.


### $ 3.2 $ Desvantagens do método de Newton

O método de Newton tem duas desvantagens significativas:
* Ele exige o cálculo da derivada da função à qual será aplicado.
* Ele nem sempre funciona, ou seja, a seqüência $ (x_n) $ pode não convergir.

Por causa do segundo defeito, uma estratégia mais adequada é utilizar o método
de Newton em combinação com um mais confiável, como o método da bissecção.
Começando com um intervalo $ [a, b] $ que encaixota um zero, utilizamos o método
de Newton começando com a estimativa inicial $ x_0 = \frac{a + b}{2} $. Mas
sempre que ocorrer
$$ \left\vert f(x_{n + 1}) \right\vert > \vert f(x_n) \vert \quad
\text{ou que} \quad x_{n + 1} \not \in [a, b]\,, $$
rejeitamos esta nova estimativa fornecida pelo método de Newton e aplicamos em
vez disto uma iteração do método da bissecção. Eventualmente $ x_n $ ficará
próximo o suficiente do zero que está sendo aproximado, e então a convergência
será rápida. Deixamos a cargo do leitor implementar o procedimento descrito.

## $ \S 4 $ Dificuldades na convergência do método de Newton: estudo de casos

É importante lembrar que, assim como para método do ponto fixo, os critérios
gerais para convergência do método de Newton são todos _locais_, isto é, sob
condições favoráveis podemos garantir a convergência a um zero $ \zeta $
contanto que a estimativa inicial $ x_0 $ seja escolhida suficientemente próxima
dele.  Vamos ilustrar através de alguns exemplos algumas dificuldades comuns que
podem surgir.

__Exemplo 2 (comportamento errático perto de extremos locais):__
Recorde que se $ a $ é um extremo (i.e., ponto de mínimo ou máximo) local, então
$ f'(a) = 0 $. Portanto, se $ x_n $ está próximo de um extremo local, então $
\left \vert f'(x_n) \right \vert $ é pequeno, logo a próxima estimativa
fornecida pode ficar muito distante da anterior. Nestes casos perdemos o
controle sobre o comportamento subseqüente.

Esta situação está ilustrada na animação abaixo, em que o método de Newton
é aplicado à função
$$
    f(x) = 2x^5 - x + 1\,.
$$

![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_2.png
"Exemplo de aplicação do método de Newton")

In [127]:
f = lambda x: 2 * x**5 - x + 1
df = lambda x: 10 * x**4 - 1
a = -1.1
b = 1.0
x_0 = 0.0
N = 4
pausa = 0
titulo = "Método de Newton para $ y = 2x^5 - x + 1 $. Perto do extremo"\
         "\n local, $ \\vert f' \\vert $ é pequeno; a estimativa é jogada para longe."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         0.00000000         1.00000000   |
|      01         1.00000000         2.00000000   |
|      02         0.77777778         0.79147826   |
|      03         0.48017397         0.57087924   |
|      04         1.69898988        27.61388409   |
|_________________________________________________|



qt.qpa.xcb: QXcbConnection: XCB error: 3 (BadWindow), sequence: 24815, resource id: 12114092, major code: 40 (TranslateCoords), minor code: 0


__Exemplo 3 (a seqüência gerada pelo método converge a um zero distinto do desejado):__
A função
$$
    f(x) = \frac{1}{1 + x^2} - \frac{1}{2}
$$
tem derivada de todas as ordens e sua primeira derivada só se anula em
$ x = 0 $. Portanto, desde que ele seja evitado, não há obstáculos para se
aplicar o método de Newton para aproximar um dos dois zeros dela, em $ x = -1 $
e em $ x = 1 $. Contudo, se tomarmos como estimativa inicial $ x_0 = 2 $ com
intenção de aproximar o zero positivo, verificaremos que a seqüência resultante
acaba convergindo ao zero negativo.

![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_3.png
"Exemplo de aplicação do método de Newton")

In [97]:
f = lambda x: 1 / (1 + x**2) - 1 / 2
df = lambda x: -2 * x * (1 + x**2)**(-2)
a = -2.2
b = 2.2
x_0 = 2
N = 7
pausa = 0.5
titulo = "Método de Newton para $ y = \\frac{1}{1 + x^2} - \\frac{1}{2} $"\
         " com $ x_0 = 2 $."\
         "\n A seqüência converge ao zero em $ x = -1 $, em vez de $ x = 1 $."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pause)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         2.00000000        -0.30000000   |
|      01         0.12500000         0.48461538   |
|      02         2.12451172        -0.31862865   |
|      03        -0.15508667         0.47651303   |
|      04        -1.76615596        -0.25724071   |
|      05        -0.53041070         0.28043572   |
|      06        -0.96443773         0.01809709   |
|      07        -0.99939056         0.00030481   |
|_________________________________________________|



__Exemplo 4 (a seqüência oscila entre dois ou mais valores sem convergir):__
Suponha que queiramos aproximar um zero da função seno. Seja $ \alpha $ um
ângulo (em radianos) tal que
$$
    \tan(\alpha) = 2\alpha \qquad (\alpha > 0)\,.
$$
Aplicando o método de Newton a $ f(x) = \sin x $ com estimativa inicial
$ x_0 = \alpha $, obtemos como próxima estimativa
$$
x_1 = \alpha - \frac{\sin \alpha}{\cos \alpha} =
\alpha - \tan \alpha = \alpha - 2 \alpha = -\alpha \,.
$$
Agora, como a tangente é uma função ímpar, 
$$
x_2 = -\alpha - \tan(-\alpha) = -\alpha + \tan(\alpha)
= -\alpha + 2\alpha = \alpha\,.
$$
Continuando, verifica-se por indução que as estimativas para
o zero oscilam entre os dois valores $ \pm \alpha $.  Isto é ilustrado na
animação abaixo.

Mais geralmente, a seqüência $ (x_n) $ gerada pelo método de
Newton pode oscilar entre qualquer número de valores, não apenas dois.

In [131]:
from numpy import sin, cos
a = -1.4
b = 1.4
N = 4
f = lambda x: sin(x)
df = lambda x: cos(x)
x_0 = 1.165561185207092
pause = 0.5
titulo = "Método de Newton para $ y = \sin x $ com estimativa "\
         "inicial infeliz.\n As aproximações $ x_n $ oscilam entre "\
         "dois valores."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pause)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         1.16556119         0.91900972   |
|      01        -1.16556119        -0.91900972   |
|      02         1.16556119         0.91900972   |
|      03        -1.16556119        -0.91900972   |
|      04         1.16556119         0.91900972   |
|_________________________________________________|



![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_4.png
"Exemplo de aplicação do método de Newton")


__Exemplo 5 (a convergência a um zero múltiplo pode ser lenta):__
A função $ f(x) = x^2 $ possui um zero de multiplicidade $ 2 $ em $ x = 0 $,
ou seja, não só vale $ f(0) = 0 $, mas também $ f'(0) = 0 $.
Neste caso, a iteração do método de Newton é dada por:
$$
x_{n + 1} = x_n - \frac{x_n^2}{2x_n} = \frac{x_n}{2}\,.
$$
Vemos portanto que os erros
$$
E_{n + 1} = 0 - x_{n + 1} \quad \text{e} \quad E_n = 0 - x_n
$$
cometidos na aproximação do zero por $ x_n $ são proporcionais, por um fator de
$ \frac{1}{2} $. Isto significa que a convergência neste caso é tão rápida (ou
lenta) quanto a do método da bisseção ou do ponto fixo.

Outra dificuldade causada pelo fato da derivada se anular no zero é que a
divisão pelo denominador $ f'(x_n) $ pequeno amplifica os erros de
arredondamento. 

Por estes motivos, _deve-se evitar o emprego do método de Newton para se
aproximar zeros de multiplicidade maior que_ $ 1 $.

In [134]:
f = lambda x: x**2
df = lambda x: 2 * x
a = -1
b = 2
x_0 = 2
N = 5
pausa = 0
N = 4

titulo = "Método de Newton para $ y = x^2 $. Aproximação mais lenta"\
         "\na um zero múltiplo e problemas com o denominador pequeno."
xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         2.00000000         4.00000000   |
|      01         1.00000000         1.00000000   |
|      02         0.50000000         0.25000000   |
|      03         0.25000000         0.06250000   |
|      04         0.12500000         0.01562500   |
|_________________________________________________|



![Exemplo de aplicação do método de Newton](fig_2-6_exemplo_5.png
"Exemplo de aplicação do método de Newton")

In [135]:
from numpy import cbrt
a = -1
b = 1.7
N = 4
f = lambda x: cbrt(x)
df = lambda x: 1 / (3 * cbrt(x**2))
x_0 = 0.1
pause = 0.5
titulo = "Falha do método de Newton para $ y = \sqrt[3]{x} $."\
         "\n A função não é diferenciável no zero em $ x = 0 $."

xs, ys = newton_animation(f, df, a, b, x_0, N, titulo, pause)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         0.10000000         0.46415888   |
|      01        -0.20000000        -0.58480355   |
|      02         0.40000000         0.73680630   |
|      03        -0.80000000        -0.92831777   |
|      04         1.60000000         1.16960710   |
|_________________________________________________|



## $ \S 5 $ Análise do erro e da convergência no método de Newton

__Teorema 5.1 (fórmula de Taylor de grau $ 1 $ com resto integral):__ _Sejam $ f $ uma função
com segunda derivada contínua num intervalo $ I $ e $ a \in I $. Então_
$$
f(x) = \underbrace{f(a) + f'(a)(x - a)}_{\substack{\text{polinômio de Taylor de grau $ 1 $} \\ \text{de $ f $ ao redor de $ a $}}} +
\underbrace{\int_a^x f''(t)(x - t)\,dt}_{\substack{\text{resto na forma
integral} \\ \text{(erro cometido na aproximação)}}} \qquad (x \in I)\,.
$$

__Prova:__ Pelo teorema fundamental do Cálculo vale
$$
f(x) = f(a) + \int_a^x f'(t)\,dt\,.
$$
Agora basta utilizar integração por partes para reescrever a integral. Tome
$$
\begin{cases}
    u = f'(t)\,, & du = f''(t)\,dt \\
    dv = dt\,, & v = -(x - t)
\end{cases}
$$
Então
\begin{alignat*}{9}
f(x) &= f(a) + \int_a^x f(t)\,dt  \\
&= f(a) + \int_a^x u\,dv \\
&= f(a) + uv \big\vert_a^x - \int_a^x v\,du \\
&= f(a) -(x - t)f'(t) \big\vert_a^x + \int_a^x f''(t)(x - t)\,dt \\
&= f(a) + (x - a)f'(a) + \int_a^x f''(t)(x - t)\,dt\,. \tag*{$ \blacksquare $}
\end{alignat*}



__Teorema 5.2 (teorema do valor médio para integrais, versão estendida):__
_Sejam $ f,\, g \colon [a, b] \to \mathbb{R} $ funções contínuas tais que $ g $
não troca de sinal em $ [a, b] $. Então_ 
$$
\int_a^b f(x)\,g(x)\,dx = f(c)\int_a^bg(x)\,dx \qquad \text{para algum } c \in (a, b)\,.
$$

__Prova:__ 
Sejam
$$
    m = \min_{[a, b]} f\quad \text{e} \quad M = \max_{[a, b]} f\,.
$$
Não há perda de generalidade ao se assumir que vale $ g \ge 0 $ em $ [a, b] $,
já que o caso em que $ g \le 0 $ pode ser reduzido a este trocando-se $ g $ por
$ -g $ (o que altera os sinais de ambos os lados da igualdade afirmada).
Então, sob esta hipótese, para qualquer $ x \in [a, b] $ temos:
\begin{equation*}
    m\,g(x) \le f(x) g(x) \le M\, g(x)\,.
\end{equation*}
Integrando, deduzimos que
\begin{equation*}
m \int_a^b g(x)\,dx \le \int_a^b f(x)g(x)\,dx \le M \int_a^b g(x)\,dx \,.
\end{equation*}

Se $ g $ é identicamente nula, a igualdade afirmada é óbvia. Caso contrário,
sua integral sobre a $ [a, b] $ é estritamente positiva. Assim, dividindo todos
os termos da desigualdade acima por
$ \int_a^b g(x)\,dx $ concluímos que
\begin{equation*}
    m \le \frac{\int_a^b g(x)f(x)\,dx}{\int_a^b g(x)\,dx} \le M .
\end{equation*}
Como $ f $ é contínua por hipótese e $ m $ e $ M $ são seu mínimo e máximo
em $ [a, b] $ respectivamente, o teorema do valor intermediário garante a
existência de um $ c \in [a, b] $ tal que o termo do meio acima é igual a $ f(c)
$. Ou seja,
\begin{equation*}
    \int_a^b f(x)\,g(x)\,dx = f(c)\int_a^bg(x)\,dx
    \qquad \text{para algum } c \in (a, b)\,. \tag*{$ \blacksquare $}
\end{equation*}


Voltando ao estudo do método de Newton, seja $ x_n $ a $n$-ésima estimativa
para um zero $ \zeta $ de $ f $. Gostaríamos de obter uma fórmula relacionando
os erros
\begin{equation*}\label{E:errors}
E_{n + 1} = \zeta - x_{n + 1} \quad \text{e} \quad E_n = \zeta - x_n\,. \tag{3}
\end{equation*}

Pela fórmula de Taylor ao redor de
$ x_n $,
\begin{equation*}\label{E:zero}
f(\zeta) = 0 = f(x_n) + (\zeta - x_n)f'(x_n) +
\int_{x_n}^\zeta (\zeta - t)f''(t)\,dt\,. \tag{1}
\end{equation*}
Por outro lado, a próxima estimativa $ x_{n + 1} $ no método de Newton é definida
implicitamente pela igualdade 
\begin{equation*}\label{E:estimate}
0 = f(x_n) + (x_{n + 1} - x_n) f'(x_n) \tag{2}
\end{equation*}
Subtraindo \eqref{E:estimate} de \eqref{E:zero} deduzimos que
$$
0 = (\zeta - x_{n + 1})f'(x_n) +
\int_{x_n}^\zeta (\zeta - t)f''(t)\,dt\,.
$$
Agora, usando o teorema do valor médio para integrais (com
$ g \leftarrow (\zeta - t) $ e $ f \leftarrow f'' $), concluímos
finalmente que
\begin{alignat*}{9}
0 &= (\zeta - x_{n + 1})f'(x_n) +
f''(c)\int_{x_n}^\zeta (\zeta - t)\,dt \\
&= \underbrace{(\zeta - x_{n + 1})}_{E_{n + 1}}\,f'(x_n) +
\frac{f''(c)}{2}\,\underbrace{(\zeta - x_n)^2}_{E_n}
\qquad \text{para algum $ c \in (a, b) $\,.}
\end{alignat*}
Daí deduzimos imediatamente a seguinte relação entre o erro atual $ E_n $ e o
próximo $ E_{n + 1} $:
$$
\boxed{E_{n + 1} = -\frac{f''(c)}{2 f'(x_n)} E_n^2}
$$
Aqui $ c $ é algum ponto no menor intervalo contendo $ \zeta $, $ x_n $ e
$ x_{n + 1} $.  Informalmente, esta fórmula significa que o próximo erro é
aproximadamente proporcional ao _quadrado_ do anterior.

__Teorema 5.3 (convergência do método de Newton):__ _Sejam $ f $ uma função
e $ \zeta $ um zero de $ f $. Suponha que:_
* $ f $ _tem segunda derivada contínua num intervalo aberto contendo $ \zeta $;_
* $ f'(\zeta) \ne 0 $.

_Então para toda estimativa inicial $ x_0 $ suficientemente próxima de $ \zeta $,
o método de Newton gera uma seqüência $ (x_n) $ que converge a $ \zeta $._

📝 Aqui "suficientemente próxima" significa que existe um $ \varepsilon > 0 $
tal que para qualquer estimativa inicial $ x_0 $ dentro do intervalo
$ J = [\zeta - \varepsilon\,,\, \zeta + \varepsilon] $, a seqüência $ (x_n) $
correspondente tem limite $ \zeta $.

__Prova:__ Seja $ I = [\zeta - \delta\,,\, \zeta + \delta] $ um intervalo
centrado em $ \zeta $ e contido no domínio de $ f $, com $ 0 < \delta < 1 $.  A
primeira derivada de $ f $ é automaticamente contínua porque existe $ f'' $.  Em
particular, reduzindo $ \delta $ se necessário podemos supor que $ f' \ne 0 $ em
$ I $. Por outro lado, a continuidade de $ f'' $ é garantida por hipótese. Logo
existem:
* $ m > 0 $ tal que $ \left\lvert f' \right \rvert > m $ em $ I $;
* $ M > 0 $ tal que $ \left\lvert f'' \right \rvert < M $ em $ I $.

Seja
$$
C = \frac{M}{m}\,.
$$
Se $ x_n $ e $ x_{n + 1} $ ambos pertencem a $ I $, vale
$$
\left\lvert E_{n + 1} \right\rvert \le
\frac{C}{2}\left\lvert E_{n} \right\rvert^2\,.
$$
Tome
$$
\varepsilon = \min\Big\{\tfrac{1}{C}\,,\,\delta \Big\} \quad \text{e}
\quad J = [\zeta - \varepsilon\,,\, \zeta + \varepsilon]\,.
$$
Então se $ x_0 \in J $, $ \vert E_0 \vert \le \varepsilon $. Logo
\begin{alignat*}{9}
\left \lvert E_1 \right \rvert &\le \frac{C}{2}\, \varepsilon^2
&\le \frac{C}{2}\frac{1}{C} \,\varepsilon
&= \frac{\varepsilon}{2}\,.
\end{alignat*}
Mais geralmente, usando o mesmo argumento prova-se por indução que
$$
\left \lvert E_n \right \rvert \le \frac{\varepsilon}{2^n}\,.
$$
Em particular, $ E_n \to 0 $ conforme $ n \to \infty $. Logo $ (x_n) $
converge a $ \zeta $ como afirmado, desde que $ x_0 \in J $.

<div style="text-align: right">$ \blacksquare $ </div>



## $ \S 7 $ Método de Newton como caso especial do método do ponto fixo

Nesta seção mostraremos que uma outra interpretação para o método de Newton é
que ele é um caso especial do método do ponto fixo. Em particular, este ponto de
vista resultará sem nenhum esforço num critério suficiente para convergência.

Suponha que $ f $ tenha derivada contínua e que $ f(\zeta) = 0 $. Seja
$ \alpha $ qualquer outra função diferenciável, mas que _não_ se anula 
num intervalo aberto $ J $ contendo $ \zeta $. Então, tomando
$$
    \varphi(x) = x + \alpha(x)f(x)\,,
$$
vale 
$$
\phantom{\qquad ( x \in I)}
f(x) = 0 \Longleftrightarrow \varphi(x) = x \qquad (x \in J)\,.
$$
Em palavras, em $ I $, os zeros de $ f $ são exatamente os pontos fixos de
$ \varphi $.

O fato que podemos transformar o problema de se encontrar zeros de $ f $ no
de se encontrar pontos fixos de outra função $ \varphi $ não é novo, já foi
discutido no caderno anterior. Lá foi visto que é possível garantir que o método
do ponto fixo aplicado a $ \varphi $ gera uma seqüência $ (x_n) $ que converge a
$ \zeta $ desde que tomemos a estimativa inicial $ x_0 $ para $ \zeta $ dentro
dum intervalo $ J \subset I $ centrado em $ \zeta $ onde valha
$$
    \vert \varphi'(x) \vert \le C \quad
    \text{para alguma constante $ C < 1 $ e todo $ x \in J $}\,.
$$
Como até agora não especificamos quem deve ser $ \alpha $, podemos escolhê-la da
maneira mais conveniente possível, a saber, de modo que a derivada de
$ \varphi $ em $ \zeta $ seja nula:
$$
\varphi'(\zeta) = 1 + \alpha(\zeta) f'(\zeta) +
\alpha'(\zeta) \underbrace{f(\zeta)}_{0} = 0\,.
$$
Gostaríamos portanto que
$$
    \alpha(\zeta) = -\frac{1}{f'(\zeta)}\,.
$$
A alternativa mais natural é definir
$$
    \alpha(x) = -\frac{1}{f'(x)}\,.
$$
Fazendo esta escolha obtemos a função de iteração
$$
    \boxed{\varphi(x) = x -\frac{f(x)}{f'(x)}}
$$
Vemos portanto que _para uma mesma estimativa inicial para $ x_0 $, a seqüência
$ (x_n) $ construída através do método de Newton aplicado a $ f $ é exatamente a
mesma que aquela construída pelo método do ponto fixo aplicado a esta função $
\varphi $._

Em particular, aplicando o Corolário 6.5 do caderno anterior a $ \varphi $ deduzimos
o seguinte critério suficiente para convergência do método de Newton.

__Teorema 7.1 (critério suficiente para convergência do método de Newton):__
_Seja $ f $ uma função continuamente diferenciável que tem um zero em $ \zeta $.
Suponha que $ I $ seja um intervalo centrado em $ \zeta $ onde valha_
\begin{equation*}
\boxed{
\left \lvert \frac{f\,f''}{\big(f'\big)^2} \right \rvert
\le C < 1}
\end{equation*}
_Então $ \zeta $ é o único zero de $ f $ em $ I $ e a seqüência $ (x_n) $ gerada
pelo método de Newton converge a $ \zeta $ para qualquer estimativa inicial
$ x_0 $ dentro de $ I $._

__Prova:__ Pela regra do quociente,
$$
\varphi'(x) = 1 - \frac{f'(x)^2 - f(x)f''(x)}{\big[f'(x)\big]^2} = 
\frac{f(x)f''(x)}{\big[f'(x)\big]^2}\,.
$$
O Corolário 6.5 se aplica desde que existam uma constante positiva $ C < 1 $ e
um intervalo $ I $ centrado em $ \zeta $ tais que 
$$
\varphi'(x) \le C \quad \text{para todo $ x \in I $}\,,
$$
ou seja,
\begin{equation*}
\left \lvert \frac{f(x)f''(x)}{\big[f'(x)\big]^2} \right \rvert
\le C \,.\tag*{$ \blacksquare $}
\end{equation*}

In [105]:

from numpy import sqrt, sign
a = -0.5
b = 0.5

f = lambda x: sign(x) * sqrt(sign(x) * x)
df = lambda x: 1 / (2 * sqrt(abs(x)))
pause = 0
x = 0.5
N = 4
titulo = "Método de Newton para $ y = \operatorname{sinal}(x)"\
         "\sqrt{\operatorname{sinal}(x)\,x} $."\
         "\n Oscilação entre dois valores."
xs, ys = newton_animation(f, df, a, b, x, N, titulo, pause)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         0.50000000         0.70710678   |
|      01        -0.50000000        -0.70710678   |
|      02         0.50000000         0.70710678   |
|      03        -0.50000000        -0.70710678   |
|      04         0.50000000         0.70710678   |
|_________________________________________________|



In [129]:

from numpy import sqrt
a = -2.2
b = 2.2

f = lambda x: sqrt(1 + x**2)
df = lambda x: x / sqrt(1 + x**2)
pause = 0.5
x = 0.5
N = 7

titulo = "Método de Newton para $ y = \\frac{1}{1 + x^2} - \\frac{1}{2} $"\
         " com $ x_0 = 2 $."\
         "\n A seqüência converge ao zero em $ x = -1 $, em vez de $ x = 1 $."
xs, ys = newton_animation(f, df, a, b, x, N, titulo, pause)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00         0.50000000         1.11803399   |
|      01        -2.00000000         2.23606798   |
|      02         0.50000000         1.11803399   |
|      03        -2.00000000         2.23606798   |
|      04         0.50000000         1.11803399   |
|      05        -2.00000000         2.23606798   |
|      06         0.50000000         1.11803399   |
|      07        -2.00000000         2.23606798   |
|_________________________________________________|



In [139]:

from numpy import sqrt
a = -1
b = 1

f = lambda x: x**5 - x + 1
df = lambda x: 5 * x**4 - 1
pause = 0.5
x = -0.0833570
N = 7

titulo = "Método de Newton para $ y = x^5 - x + 1 $ exibindo"\
         "\noscilação entre três valores."
xs, ys = newton_animation(f, df, a, b, x, N, titulo, pause)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|      00        -0.08335700         1.08335298   |
|      01         1.00025756         1.00103091   |
|      02         0.75032183         0.48749244   |
|      03        -0.08335712         1.08335309   |
|      04         1.00025756         1.00103091   |
|      05         0.75032183         0.48749244   |
|      06        -0.08335710         1.08335307   |
|      07         1.00025756         1.00103091   |
|_________________________________________________|

